In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## World Bank's Poverty Reduction 10-year Program

As stated on the report, the main objective of this analysis is to generate insights that will help the World Bank's Financial Management and Loan Comitee areas to design a 10 year strategy for poverty reduction in the African continent. The analysis and content of this file can be briefly summarized in the following three areas which have been explained in detail in the written report that complements this analysis:

- Data Understanding and preparation, relevant feature selection and data cleaning.
- Overall business understanding. How much has the World Bank awarded on the last years? To which regions?
- Contracts awarded specifically to the African continent in the last years. Which cities and sectors have benefited?


The CSV file that has been used includes data on contracts for Investment Projects that were reviewed and awarded by the World Bank as of September 2017. This dataset does not list all contracts awarded by the Bank, but should be viewed  as a guide to determine and analyze the distribution of Major contract commitments among the Bank's member countries. 
Click <a href="https://www.kaggle.com/theworldbank/world-banks-major-contracts">here</a> to go to the website where you can download the original version of the file.

The dataset is composed of the following variables (note that some of them are for banks internal use and provide no value for our analysis):

1. **As of Date:** 6-Sep-17, date when the file was generated.
2. **Fiscal Year:** fiscal year when the actual loan has been implemented.
3. **Region:** geographical area from the country for which the contract was awarded.
4. **Borrower Country:** country for which the contract was awarded.
5. **Borrower Country Code:** country code for which the contract was awarded.
6. **Project ID:** project's internal registry number.
7. **Project Name:** investment project registered name.
8. **Procurement Type:** specific area for which the loan will be utilized.
9. **Procurement Category:** broader category in which the loan will be utilized.
10. **Procurement Method:** type of bidding and selection.
11. **Product Line:** internal are in which the contract has been granted.
12. **Major Sector:** economic sector to be supported by the contract implementation.
13. **WB Contract Number:** contract's internal registry number.
14. **Contract Description:** short description of each contract.
15. **Contract Signing Date:** date when the contract has been formally agreed.
16. **Supplier:** primary supplier working on the project.
17. **Supplier Country:** country of supplier registration.
18. **Supplier Country Code:** country code of supplier registration.
19. **Total Contract Amount (USD):** total worth of the contract in USD.
20. **Borrower Contract Reference Number:** borrower country contract number.



#### Reading the CSV file, which contains a header, setting the Infer Schema option as True:

In [3]:
WBContracts = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("WBContracts.csv")

### 1. Data Understanding:
First actions are aimed to gather overall context and information on the data:

In [4]:
WBContracts.printSchema()

root
 |-- As of Date: string (nullable = true)
 |-- Fiscal Year: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Borrower Country: string (nullable = true)
 |-- Borrower Country Code: string (nullable = true)
 |-- Project ID: string (nullable = true)
 |-- Project Name: string (nullable = true)
 |-- Procurement Type: string (nullable = true)
 |-- Procurement Category: string (nullable = true)
 |-- Procurement Method: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Major Sector: string (nullable = true)
 |-- WB Contract Number: integer (nullable = true)
 |-- Contract Description: string (nullable = true)
 |-- Contract Signing Date: string (nullable = true)
 |-- Supplier: string (nullable = true)
 |-- Supplier Country: string (nullable = true)
 |-- Supplier Country Code: string (nullable = true)
 |-- Total Contract Amount (USD): string (nullable = true)
 |-- Borrower Contract Reference Number: string (nullable = true)



#### We can see that the columns Fiscal Year and Total Contract Amount have been imported as String, and at some point we will need to change them to Integers since we will need to perform operations with them.

In [5]:
cols = len(WBContracts.columns)
print("The World Bank Contracts dataset has", cols, "columns")
rows = WBContracts.count()
print("The World Bank Contracts dataset has", rows, "rows")

The World Bank Contracts dataset has 20 columns
The World Bank Contracts dataset has 154811 rows


Renaming some of the Features to a shorter and more user friendly Column Name, so that we can call them easily for further operations:

In [6]:
WBContracts2 = WBContracts.withColumnRenamed("Fiscal Year","Year")\
           .withColumnRenamed("Borrower Country","Country")\
           .withColumnRenamed("Procurement Category","Category")\
           .withColumnRenamed("Major Sector","Sector")\
           .withColumnRenamed("Total Contract Amount (USD)","Total_Amount")

In [7]:
WBContracts2.show(3)

+--------------------+----+------+-------+---------------------+----------+--------------------+-------------------+-------------------+--------------------+------------+--------------------+------------------+--------------------+---------------------+------------+----------------+---------------------+------------+----------------------------------+
|          As of Date|Year|Region|Country|Borrower Country Code|Project ID|        Project Name|   Procurement Type|           Category|  Procurement Method|Product line|              Sector|WB Contract Number|Contract Description|Contract Signing Date|    Supplier|Supplier Country|Supplier Country Code|Total_Amount|Borrower Contract Reference Number|
+--------------------+----+------+-------+---------------------+----------+--------------------+-------------------+-------------------+--------------------+------------+--------------------+------------------+--------------------+---------------------+------------+----------------+---------

We can see that the column Total_Amount contains a special character "$" at the beginning of each value. We need to remove it in order to be able to perform operations with it. Additionally, we will modify the data type of the previously mentioned columns to Integer:

In [8]:
import pyspark.sql.functions as f
df = WBContracts2.withColumn("Total_Amount", f.regexp_replace(f.col("Total_Amount"), "\$", ""))

In [9]:
from pyspark.sql.types import IntegerType

WBContracts3 = df.withColumn("YearTemp", df.Year.cast(IntegerType()))\
    .withColumn("TATemp", df.Total_Amount.cast(IntegerType()))\
    .drop("Year")\
    .drop("Total_Amount")\
    .withColumnRenamed("YearTemp", "Year")\
    .withColumnRenamed("TATemp", "TotalAmount")

#### WBAnalyze is for now, the final version of the DataFrame that we will be using and storing in cache memory to perform faster queries and operations:

In [10]:
WBAnalyze = WBContracts3.select("Year", "Region", "Country", "Category", "Sector", "Supplier Country Code", "TotalAmount")

WBAnalyze.cache()

DataFrame[Year: int, Region: string, Country: string, Category: string, Sector: string, Supplier Country Code: string, TotalAmount: int]

In [11]:
WBAnalyze.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Supplier Country Code: string (nullable = true)
 |-- TotalAmount: integer (nullable = true)



### 2. Initial Analysis: 
We will start by gathering some basic information from our Dataset to gain a deeper understanding of it:

1. What is the total number of loans that have been awarded by the World Bank between 2004 and 2008?
2. What is the Average amount from all the loans awarded in the same time period?
3. What are the minimun and maximum amounts of the awarded contracts?

In [12]:
WBAnalyze.describe("TotalAmount").show()

+-------+------------------+
|summary|       TotalAmount|
+-------+------------------+
|  count|            147959|
|   mean|1147646.8110219722|
| stddev|  8895662.08531883|
|    min|             -2158|
|    max|        1100669377|
+-------+------------------+



4. What is the total amount in USD that the World Bank has loaned from 2004 to 2018?

In [13]:
WBAnalyze.agg(f.sum("TotalAmount")).show()

+----------------+
|sum(TotalAmount)|
+----------------+
|    169804674512|
+----------------+



5. What is the total amount in USD that the World Bank has loaned on each year from 2004 to 2018?
   Which are the years with the highest and lowest loans?

In [14]:
WBAnalyze.groupBy("Year").sum("TotalAmount").show()

+----+----------------+
|Year|sum(TotalAmount)|
+----+----------------+
|2007|      9931846617|
|2018|       813062252|
|2015|     11760574689|
|2006|      8057177769|
|2013|     14836428497|
|null|            null|
|2014|     14187169010|
|2004|      8788488716|
|2012|     15914796289|
|2009|     10936652615|
|2016|     13853670627|
|2005|      9454535318|
|2010|     12684114826|
|2011|     15391369323|
|2008|     12492580431|
|2017|     10702207533|
| 788|            null|
+----+----------------+



6. From the previous query, we can see that we have some Null and incorrect values (788) for the variable Year. We want to know how many do we have, to asses the feasibility of removing those rows.

In [15]:
WBAnalyze.groupBy("Year").count().orderBy("Year").show()

+----+-----+
|Year|count|
+----+-----+
|null| 3187|
| 788|    1|
|2004|14513|
|2005|14281|
|2006|11858|
|2007| 9245|
|2008| 9192|
|2009|10149|
|2010|12023|
|2011|12243|
|2012|12907|
|2013|12313|
|2014|12016|
|2015|10169|
|2016| 6787|
|2017| 3640|
|2018|  287|
+----+-----+



In [16]:
WBAnalyze\
        .where("Year = 788 or Year is null")\
        .groupBy("Year").count().show()

+----+-----+
|Year|count|
+----+-----+
|null| 3187|
| 788|    1|
+----+-----+



We can see that the number of Null and incorrect values is not high (~ 2%). Therefore I have decided to remove them to get a clear picture of the total Number and Total Amount of Loans Awarded on each Year:

In [17]:
WBClean = WBAnalyze\
            .dropna(subset=["Year"])\
            .where("Year != 788")

In [18]:
WBClean.groupBy("Year").count().orderBy("Year").show()

+----+-----+
|Year|count|
+----+-----+
|2004|14513|
|2005|14281|
|2006|11858|
|2007| 9245|
|2008| 9192|
|2009|10149|
|2010|12023|
|2011|12243|
|2012|12907|
|2013|12313|
|2014|12016|
|2015|10169|
|2016| 6787|
|2017| 3640|
|2018|  287|
+----+-----+



7. What is the list of geographical Regions in which the loans have been granted worldwide?

In [19]:
WBClean.select("Region").distinct().show(20,False)

+----------------------------+
|Region                      |
+----------------------------+
|LATIN AMERICA AND CARIBBEAN |
|SOUTH ASIA                  |
|OTHER                       |
|AFRICA                      |
|MIDDLE EAST AND NORTH AFRICA|
|EAST ASIA AND PACIFIC       |
|EUROPE AND CENTRAL ASIA     |
+----------------------------+



8. What is the total amount in USD that the World Bank has awarded for each Region? Which are the Regions with the highest and lowest amounts?

In [20]:
WBClean\
        .groupBy("Region")\
        .sum("TotalAmount").withColumnRenamed("sum(TotalAmount)","SumPerRegion").orderBy("SumPerRegion",ascending=False)\
        .show(20,False)

+----------------------------+------------+
|Region                      |SumPerRegion|
+----------------------------+------------+
|AFRICA                      |42351174243 |
|SOUTH ASIA                  |33324159096 |
|EAST ASIA AND PACIFIC       |30746079077 |
|EUROPE AND CENTRAL ASIA     |28299487362 |
|LATIN AMERICA AND CARIBBEAN |23974043540 |
|MIDDLE EAST AND NORTH AFRICA|11101262642 |
|OTHER                       |8468552     |
+----------------------------+------------+



In [21]:
WBClean.groupBy("Region").count().orderBy("count",ascending=False).show(20,False)

+----------------------------+-----+
|Region                      |count|
+----------------------------+-----+
|AFRICA                      |39926|
|EUROPE AND CENTRAL ASIA     |30458|
|LATIN AMERICA AND CARIBBEAN |30088|
|EAST ASIA AND PACIFIC       |23617|
|SOUTH ASIA                  |19253|
|MIDDLE EAST AND NORTH AFRICA|8206 |
|OTHER                       |75   |
+----------------------------+-----+



### 3. African Region: 
We can see that Africa is the region with both the highest count and highest amount of loans awarded. As stated before, from now on we will focus our analysis on this region, as it is considered to be the poorest continent in the world and therefore, it is the main focus of the World Bank's project.

9. We will create a new Dataframe specificly to study and generate insights regarding loans awarded to the African Continent in the last 10 years.

In [22]:
AfricaDF = WBClean\
            .filter(f.col("Region") == 'AFRICA')\
            .filter(f.col("Year") > '2008')\
            .select("Year","Country","Category","Sector","TotalAmount")

In [23]:
AfricaDF.show(10,False)
AfricaDF.cache()

+----+---------------------+-------------------+-------------------------------------------+-----------+
|Year|Country              |Category           |Sector                                     |TotalAmount|
+----+---------------------+-------------------+-------------------------------------------+-----------+
|2009|Africa               |CONSULTANT SERVICES|Public Administration                      |184365     |
|2009|Burkina Faso         |CONSULTANT SERVICES|Public Administration                      |258926     |
|2009|Guinea               |GOODS              |Public Administration                      |468583     |
|2009|Madagascar           |CONSULTANT SERVICES|Health                                     |54788      |
|2009|Madagascar           |CONSULTANT SERVICES|Social Protection                          |101750     |
|2009|Sao Tome and Principe|GOODS              |Public Administration                      |945657     |
|2009|Tanzania             |CONSULTANT SERVICES|Water, 

DataFrame[Year: int, Country: string, Category: string, Sector: string, TotalAmount: int]

10. What is the total amount in USD that the World Bank has loaned to the Region Africa on each of the last 10 years? Which are the years with the highest and lowest loans?

In [24]:
AfricaDF\
        .groupBy("Year")\
        .sum("TotalAmount")\
        .orderBy("Year").show()

+----+----------------+
|Year|sum(TotalAmount)|
+----+----------------+
|2009|      2746476663|
|2010|      2937655252|
|2011|      5255941408|
|2012|      4947060538|
|2013|      3676230363|
|2014|      3771698114|
|2015|      2946419719|
|2016|      2861158986|
|2017|      3187819310|
|2018|       302344227|
+----+----------------+



In [25]:
AfricaDF\
        .groupBy("Year")\
        .sum("TotalAmount").withColumnRenamed('sum(TotalAmount)', 'MaxValue')\
        .orderBy("MaxValue", ascending=False).show(1)

+----+----------+
|Year|  MaxValue|
+----+----------+
|2011|5255941408|
+----+----------+
only showing top 1 row



In [26]:
AfricaDF\
        .groupBy("Year")\
        .sum("TotalAmount").withColumnRenamed('sum(TotalAmount)', 'MinValue')\
        .orderBy("MinValue", ascending=True).show(1)

+----+---------+
|Year| MinValue|
+----+---------+
|2018|302344227|
+----+---------+
only showing top 1 row



11. How many loans have been awarded per year?

In [27]:
AfricaDF.groupBy("Year").count().orderBy("Year").show()

+----+-----+
|Year|count|
+----+-----+
|2009| 2757|
|2010| 2754|
|2011| 2742|
|2012| 3219|
|2013| 3127|
|2014| 3516|
|2015| 2621|
|2016| 1652|
|2017| 1272|
|2018|  181|
+----+-----+



12. We can see that the amount and count of loans for 2018 is considerably lower. This is because the dataset has been generated on that year and the data does not cover the complete time frame for it. In order to compare 2018 with the rest of the years in a different way, we will calculate the Average amount per granted loan on each year:

In [28]:
AfricaDF.groupBy("Year").agg(f.mean("TotalAmount").alias("AvgAmount"))\
           .orderBy("AvgAmount",ascending=False).show(10,False)

+----+------------------+
|Year|AvgAmount         |
+----+------------------+
|2017|2528008.969072165 |
|2011|1984872.1329305137|
|2016|1785991.8764044943|
|2018|1670410.0939226518|
|2012|1621987.0616393443|
|2013|1248295.5392190153|
|2015|1160464.6392280424|
|2014|1105421.4871043377|
|2010|1089230.7200593252|
|2009|1012339.352377442 |
+----+------------------+



13. What is the list of different Categories for which the loans are granted in this continent?

In [29]:
AfricaDF.select("Category").distinct().show(20, False)

+-----------------------+
|Category               |
+-----------------------+
|GOODS                  |
|Not assigned           |
|CIVIL WORKS            |
|CONSULTANT SERVICES    |
|NON-CONSULTING SERVICES|
+-----------------------+



14. What is the total amount in USD that has been granted for each Category in the last 10 years?

In [30]:
AfricaDF.groupBy("Category").sum("TotalAmount").show(20,False)

+-----------------------+----------------+
|Category               |sum(TotalAmount)|
+-----------------------+----------------+
|GOODS                  |6750654346      |
|Not assigned           |2177233685      |
|CIVIL WORKS            |17211096942     |
|CONSULTANT SERVICES    |6100963189      |
|NON-CONSULTING SERVICES|392856418       |
+-----------------------+----------------+



15. What are the 10 African countries that have received the highest amount in USD as loans in the last 10 years?

In [31]:
AfricaDF.groupBy("Country")\
           .agg(f.sum("TotalAmount")\
                .alias("TotalSum"))\
           .orderBy("TotalSum",ascending=False).show(10,False)

+-----------------------------+----------+
|Country                      |TotalSum  |
+-----------------------------+----------+
|Africa                       |3827720124|
|Ethiopia                     |3540487927|
|South Africa                 |3053722422|
|Nigeria                      |2793922760|
|Kenya                        |2315083923|
|Congo, Democratic Republic of|1829425634|
|Western Africa               |1816948253|
|Tanzania                     |1238251951|
|Mozambique                   |1033955286|
|Uganda                       |930474147 |
+-----------------------------+----------+
only showing top 10 rows



16. What are the 10 African countries that have received the highest number of loans in the last 10 years?

In [32]:
AfricaDF\
        .select("Country","Category")\
        .groupBy("Country").count().orderBy('count',ascending=False).show(10,False)

+-----------------------------+-----+
|Country                      |count|
+-----------------------------+-----+
|Africa                       |1936 |
|Congo, Democratic Republic of|1801 |
|Cote d'Ivoire                |1233 |
|Nigeria                      |1222 |
|Western Africa               |1115 |
|Liberia                      |1046 |
|Madagascar                   |909  |
|Kenya                        |905  |
|Uganda                       |898  |
|Benin                        |877  |
+-----------------------------+-----+
only showing top 10 rows



17. Lastly, we will generate a new column that will assign a Numeric value to each of the values in the Category variable. This encoding will help us in the future when we train a model to generate insights and classify the different types of loans automatically:

In [33]:
AfricaM = AfricaDF.withColumn("NumCategory", f.when(f.col("Category") == "GOODS", 1)\
                                                           .when(f.col("Category") == "CIVIL WORKS", 2)\
                                                           .when(f.col("Category") == "CONSULTANT SERVICES", 3)\
                                                           .when(f.col("Category") == "NON-CONSULTING SERVICES", 4)\
                                                           .otherwise(0))
AfricaM.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- TotalAmount: integer (nullable = true)
 |-- NumCategory: integer (nullable = false)



In [34]:
AfricaM.show()

+----+--------------------+-------------------+--------------------+-----------+-----------+
|Year|             Country|           Category|              Sector|TotalAmount|NumCategory|
+----+--------------------+-------------------+--------------------+-----------+-----------+
|2009|              Africa|CONSULTANT SERVICES|Public Administra...|     184365|          3|
|2009|        Burkina Faso|CONSULTANT SERVICES|Public Administra...|     258926|          3|
|2009|              Guinea|              GOODS|Public Administra...|     468583|          1|
|2009|          Madagascar|CONSULTANT SERVICES|              Health|      54788|          3|
|2009|          Madagascar|CONSULTANT SERVICES|   Social Protection|     101750|          3|
|2009|Sao Tome and Prin...|              GOODS|Public Administra...|     945657|          1|
|2009|            Tanzania|CONSULTANT SERVICES|Water, Sanitation...|     354833|          3|
|2009|              Zambia|CONSULTANT SERVICES|Agriculture, Fish...|  

Our DataFrame (AfricaM) for training a model is now ready to use!